In [2]:
#after changes in the script using Jupyter notebook, save, and run this:
get_ipython().system(u'jupyter nbconvert --to=python integrate_CO2flux.ipynb')

[NbConvertApp] Converting notebook integrate_CO2flux.ipynb to python
[NbConvertApp] Writing 3099 bytes to integrate_CO2flux.py


In [9]:
    def integrate_CO2flux_roeden(datam,region,xax,yax):
        """
        this function integrates a monthly 3d field of CO2 flux into an annual mean time-series, over specific regions

        """
      
        import numpy as np
        from annualmean import annualmean
        import matplotlib.pyplot as plt
        import matplotlib.cm as cm
        cmap = cm.jet
        cmap.set_bad('white',1.)
        
        nn = np.shape(datam)
        
        nt = nn[0]
        nx=nn[2]
        ny=nn[1]
        
        #print np.shape(yax)
        yax2=yax[np.newaxis,:,np.newaxis]
        #print np.shape(yax2)
        YY = np.tile(yax2,(nt,1,nx))
        
#        print np.shape(YY)
               
        # define indices for regions
        # define areas
        

        maskG = (YY[:] < 90)
        maskN = ((YY[:]>30))
        maskS = (YY[:]<-30)
        maskT = ((YY[:]>-30) & (YY[:] < 30))

            

        # define right region index
        if region == 'global':
            
            mask = maskG
       # elif self.region == 'pac': #do not filter more, indR should be global/all
       #     indR = glob
        elif region == 'S':
            mask = maskS
        elif region == 'N':
            mask = maskN
        elif region == 'T':
            mask = maskT
        else:
            raise NameError('Region not defined properly')
            
        #unit is molC/m2/s, convert to PgC/yr
        # *12: mol --> gC, *1e-15: g --> Pg, *86400*365.25: /s -> /yr
        conv=12*1e-15 *86400*365.25
 
       # print np.shape(datam)
    
        # initialize array full of NaNs
        datam2 = np.full_like(datam, np.nan)
        
        # fill with values of region to be used, rest stays NaN 
        datam2[mask] = datam[mask]
        data_m = np.ma.array(datam2, mask=np.isnan(datam2))
    
        tmp2 = data_m * areaT   
    
    # print np.shape(tmp2)
        # sum up and convert units
        #sum first over xax, then yax, keep timesteps (axis0)
        tmp3 = conv*(np.nansum(np.nansum(tmp2,axis=2),axis=1))
      #  print np.shape(tmp3)
        fgco2_global=tmp3
        # annual_mean
        fgco2_global_am=annualmean(tmp3)
      #  print np.shape(fgco2_global_am) 
        
        #########################################   
        ### quickly check fields:
        
    #    lon, lat = np.meshgrid(xax,yax, indexing='ij')

        #print np.shape(np.squeeze(datam[:,indR,:]))
        #graphic models
    #    print "plotting year 2015 December"
    #    fig_mod = plt.figure(1,figsize=(10,10))
    #    plt.pcolor(xax,yax,np.squeeze(data_m[-25,:,:]), vmin=-2e-7, vmax=5e-7, cmap=cmap)#vmin=np.nanmin(x), vmax=np.nanmax(x)) 
    #    plt.colorbar()
    #    plt.show()
        return fgco2_global_am
    
        
    

#test
import numpy as np
from netCDF4 import Dataset
gcb_data = Dataset('../Output/gcb_load.nc')
roeden_spco2_1x1 = gcb_data['roeden_spco2_1x1'][:]

mean = annualmean(roeden_spco2_1x1)
print np.shape(roeden_spco2_1x1), np.shape(mean)